# FLOPS demo

In [1]:
:opt no-lint -- turn off lint messages

## module PiCalc

In [2]:
{-# LANGUAGE FlexibleContexts          #-}
{-# LANGUAGE FlexibleInstances         #-}
{-# LANGUAGE MultiParamTypeClasses     #-}
{-# LANGUAGE NoMonomorphismRestriction #-}
{-# LANGUAGE ScopedTypeVariables       #-}
{-# LANGUAGE DeriveGeneric             #-}
{-# LANGUAGE UndecidableInstances      #-}

module PiCalc where
import GHC.Generics (Generic)
import Data.Maybe
import Data.List
import Unbound.Generics.LocallyNameless

type Nm = Name Tm
newtype Tm = Var Nm deriving (Eq, Ord, Show, Generic)

data Pr  = Null | TauP Pr | Out Tm Tm Pr | In Tm PrB | Match Tm Tm Pr
         | Plus Pr Pr | Par Pr Pr | Nu PrB  deriving (Eq, Ord, Show, Generic)
type PrB = Bind Nm Pr
instance Eq PrB where (==) = aeq
instance Ord PrB where compare = acompare

data Act   = Up Tm Tm  | Tau     deriving (Eq, Ord, Show, Generic)
data ActB  = UpB Tm    | DnB Tm  deriving (Eq, Ord, Show, Generic)

data Form  = FF | TT | Conj [Form] | Disj [Form]
           | Dia  Act Form  |  DiaB  ActB FormB   | DiaMatch [(Tm,Tm)]
           | Box  Act Form  |  BoxB  ActB FormB   | BoxMatch [(Tm,Tm)] Form
           deriving (Eq, Ord, Show, Generic)
type FormB = Bind Nm Form
instance Eq FormB where (==) = aeq
instance Ord FormB where compare = acompare

instance Alpha Tm; instance Alpha Act; instance Alpha ActB
instance Alpha Pr; instance Alpha Form

instance Subst Tm Tm where isvar (Var x) = Just (SubstName x)
instance Subst Tm Act;  instance Subst Tm ActB
instance Subst Tm Pr;  instance Subst Tm Form

infixr 1 .\
(.\) = bind

x .= y = Match (Var x) (Var y)
inp = In . Var
out x y = Out(Var x)(Var y)
tau = TauP Null
tautau = TauP (TauP Null)

conj  = cn . filter(/=TT) where cn  [] = TT; cn  [f] = f; cn  fs = Conj fs
disj  = ds . filter(/=FF) where ds  [] = FF; ds  [f] = f; ds  fs = Disj fs

unbind2' b1 b2 = do  Just (x,p1,_,p2) <- unbind2 b1 b2
                     return (x,p1,p2)
(.+)  = Plus  ;   infixl 6 .+
(.|)  = Par   ;  infixl 5 .|
o = Null
taup = TauP
nu = Nu

## module MemoUgly

In [3]:
{-# LANGUAGE NoMonomorphismRestriction #-}
module MemoUgly where
import Control.Concurrent.MVar
import qualified Data.Map as M
import System.IO.Unsafe(unsafePerformIO)
import qualified Data.MemoUgly

memo = Data.MemoUgly.memo
memoIO = Data.MemoUgly.memoIO

memoFix ff = f where f = memo (ff f)

## module IdSubLTS

In [4]:
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE FlexibleInstances #-}
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE UndecidableInstances #-}
{-# LANGUAGE NoMonomorphismRestriction #-}

module IdSubLTS where

import Control.Applicative
import Control.Lens.Fold
import Control.Monad
import Control.Monad.Fail
import Control.Monad.Trans.Identity
import PiCalc
import Unbound.Generics.LocallyNameless hiding (fv)
import qualified Unbound.Generics.LocallyNameless as U


fv = toListOf U.fv

interactsB (UpB x) (DnB x') = x == x'
interactsB (DnB x) (UpB x') = x == x'
interactsB _ _ = False

one :: (Alternative m, Fresh m, MonadFail m) => Pr -> m (Act,Pr)
one (Out x y p) = return (Up x y, p)
one (TauP p)    = return (Tau, p)
one (Match x y p) =
      do guard $ x == y
         one p
one (Plus p q) = one p <|> one q
one (Par p q) =
      do (l, p') <- one p; return (l, Par p' q)
  <|> do (l, q') <- one q; return (l, Par p q')
  <|> do (lp, bp) <- oneb p
         (lq, bq) <- oneb q
         guard $ interactsB lp lq -- close
         (y, p', q') <- unbind2' bp bq
         return (Tau, Nu (y .\ Par p' q'))
  <|> do (Up x v, p') <- one p
         (DnB x', (y, q')) <- oneb' q
         guard $ x == x'
         return (Tau, Par p' (subst y v q')) -- interaction
  <|> do (DnB x', (y, p')) <- oneb' p
         (Up x v, q') <- one q
         guard $ x == x'
         return (Tau, Par (subst y v p') q') -- interaction
one (Nu b) =
      do (x, p) <- unbind b
         (l, p') <- one p
         guard $ x `notElem` fv l
         return (l, Nu (x .\ p'))
one _ = empty

oneb :: (Alternative m, Fresh m, MonadFail m) => Pr -> m (ActB, PrB)
oneb (In x p) = return (DnB x, p)
oneb (Match x y p) =
      do guard $ x == y
         oneb p
oneb (Plus p q) = oneb p <|> oneb q
oneb (Par p q) =
      do (l, (x, p')) <- oneb' p; return (l, x .\ Par p' q)
  <|> do (l, (x, q')) <- oneb' q; return (l, x .\ Par p q')
oneb (Nu b) =
      do (x, p) <- unbind b
         (l, (y, p')) <- oneb' p
         guard $ x `notElem` fv l
         return (l, y .\ Nu (x .\ p'))
  <|> do (x, p) <- unbind b
         (l@(Up (Var y) (Var x')), p') <- one p
         guard $ x /= y && x == x'
         return (UpB (Var y), x .\ p') -- open
oneb _ = empty

oneb' p = do (l, b) <- oneb p; r <- unbind b; return (l, r)
{-
% Finite pi-calculus specification in lambda-Prolog
% A specification of the late transition system for the finite pi calculus.
% bound input
oneb (in X M) (dn X) M.
% free output
one (out X Y P) (up X Y) P.
% tau
one  (taup P) tau P.
% match prefix
one  (match X X P) A Q :- one  P A Q.
oneb (match X X P) A M :- oneb P A M.
% sum
one  (plus P Q) A R :- one  P A R.
one  (plus P Q) A R :- one  Q A R.
oneb (plus P Q) A M :- oneb P A M.
oneb (plus P Q) A M :- oneb Q A M.
% par
one  (par P Q) A (par P1 Q) :- one P A P1.
one  (par P Q) A (par P Q1) :- one Q A Q1.
oneb (par P Q) A (x\par (M x) Q) :- oneb P A M.
oneb (par P Q) A (x\par P (N x)) :- oneb Q A N.
% restriction
one  (nu x\P x) A (nu x\Q x)      :- pi x\ one  (P x) A (Q x).
oneb (nu x\P x) A (y\ nu x\Q x y) :- pi x\ oneb (P x) A (y\ Q x y).
% open
oneb (nu x\M x) (up X) N :- pi y\ one (M y) (up X y) (N y).
% close
one (par P Q) tau (nu y\ par (M y) (N y)) :- oneb P (dn X) M , oneb Q (up X) N.
one (par P Q) tau (nu y\ par (M y) (N y)) :- oneb P (up X) M , oneb Q (dn X) N.
% comm
one (par P Q) tau (par (M Y) T) :-  oneb P (dn X) M, one Q (up X Y) T.
one (par P Q) tau (par R (M Y)) :-  oneb Q (dn X) M, one P (up X Y) R.
-}


## module OpenLTS

In [5]:
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE FlexibleInstances #-}
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE NoMonomorphismRestriction #-}
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE UndecidableInstances #-}
{-# LANGUAGE DeriveGeneric #-}

module OpenLTS where

import GHC.Generics (Generic)
import Control.Applicative
import Control.Lens.Fold
import Control.Monad
import Control.Monad.Fail
import Control.Monad.Trans.Reader
import Data.Partition hiding (empty,rep)
import Data.List
import qualified Data.Set as S
import Data.Maybe
import qualified Data.Partition as P
import PiCalc
import Unbound.Generics.LocallyNameless hiding (fv)
import qualified Unbound.Generics.LocallyNameless as U

fv = toListOf U.fv

type Ctx = [Quan]

data Quan = All Nm | Nab Nm deriving (Eq,Ord,Show,Generic)

quan2nm :: Quan -> Nm
quan2nm (All x) = x
quan2nm (Nab x) = x

alls qs = [x | All x <- qs]
nabs qs = [x | Nab x <- qs]

instance Alpha Quan
instance Subst Tm Quan

type Constraint = Partition Int

part2NmSets sigma = do
  xs <- reversedCtxNames
  return $ S.map (xs!!) <$> P.nontrivialSets sigma

respects :: Constraint -> [Int] -> Bool
respects part ns = (P.rep part <$> ns) == ns -- ns - indices of Nab names

respectful sigma =
  do ns <- mapM indexNm . nabs =<< ask
     guard $ respects sigma ns
     return sigma

reversedCtxNames = reverse <$> ctxNames

ctxNames = asks (map quan2nm)

indexNm :: Monad m => Nm -> ReaderT Ctx m Int
indexNm x = fromJust . elemIndex x <$> reversedCtxNames

-- error when x is not in ctx
-- calling reverse every time is not efficient - refactor later
indexNmWith :: Monad m => Constraint -> Nm -> ReaderT Ctx m Int
indexNmWith sigma x = P.rep sigma <$> indexNm x

joinNm :: MonadPlus m => Nm -> Nm -> ReaderT Ctx m Constraint
joinNm x y = respectful =<< joinElems <$> indexNm x <*> indexNm y <*> pure P.discrete

joinTm :: MonadPlus m => Tm -> Tm -> ReaderT Ctx m Constraint
joinTm (Var x) (Var y) = joinNm x y

joinParts = respectful . fromSets . concatMap nontrivialSets

elemWith sigma x xs =
  do i <- indexNmWith sigma x
     is <- mapM (indexNmWith sigma) xs
     guard $ i `elem` is

notElemWith sigma x xs =
  do i <- indexNmWith sigma x
     is <- mapM (indexNmWith sigma) xs
     guard $ i `notElem` is

equalWith sigma x y = elemWith sigma x [y]
noteqWith sigma x y = notElemWith sigma x [y]

interactsB (UpB x) (DnB x') = joinTm x x'
interactsB (DnB x) (UpB x') = joinTm x x'
interactsB _ _ = empty

extendCtx = (:)

subs ctx sigma = substs [(x, Var y) | i <-[0..length ns-1],
                                       let x = ns !! i,
                                       let y = ns !! P.rep sigma i ]
  where ns = reverse . map quan2nm $ ctx

one (Out x y p) = return (P.empty,(Up x y,p))
one (TauP p)    = return (P.empty,(Tau,p))
one (Match x y p) =
      do sigmaxy <- joinTm x y
         (sigma,r) <- one p
         sigma' <- joinParts [sigmaxy,sigma]
         return (sigma',r)
one (Plus p q) = one p <|> one q
one (Par p q) =
      do (sigma,(l,p')) <- one p; return (sigma,(l,Par p' q))
  <|> do (sigma,(l,q')) <- one q; return (sigma,(l,Par p q'))
  <|> do (sigma_p,(lp,bp)) <- oneb p
         (sigma_q,(lq,bq)) <- oneb q
         sigma <- interactsB lp lq             -- close
         sigma' <- joinParts [sigma,sigma_p,sigma_q]
         (y,p',q') <- unbind2' bp bq
         return (sigma',(Tau,Nu (y .\ Par p' q')))
  <|> do (sigma_p,(Up x v,p')) <- one p
         (sigma_q,(DnB x',(y,q'))) <- oneb' q
         sigma <- joinTm x x'
         sigma' <- joinParts [sigma,sigma_p,sigma_q]
         return (sigma',(Tau,Par p' (subst y v q')))  -- interaction
  <|> do (sigma_p,(DnB x',(y,p'))) <- oneb' p
         (sigma_q,(Up x v,q')) <- one q
         sigma <- joinTm x x'
         sigma' <- joinParts [sigma,sigma_p,sigma_q]
         return (sigma',(Tau,Par (subst y v p') q'))  -- interaction
one (Nu b) =
      do (x,p) <- unbind b
         (sigma,(l,p')) <- local (extendCtx (Nab x)) $
            do ret@(sigma,(l,p')) <- one p
               notElemWith sigma x (fv l)
               return ret
         return (sigma,(l,Nu (x .\ p')))
one _ = empty

oneb (In x p) = return (P.empty,(DnB x,p))
oneb (Match x y p) =
      do sigmaxy <- joinTm x y
         (sigma,r) <- oneb p
         sigma' <- joinParts [sigmaxy,sigma]
         return (sigma',r)
oneb (Plus p q) = oneb p <|> oneb q
oneb (Par p q) =
      do (sigma,(l,(x,p'))) <- oneb' p; return (sigma,(l,x .\ Par p' q))
  <|> do (sigma,(l,(x,q'))) <- oneb' q; return (sigma,(l,x .\ Par p q'))
oneb (Nu b) =
      do (x,p) <- unbind b
         (sigma,(l,(y,p'))) <- local (extendCtx (Nab x)) $
           do ret@(sigma,(l,r)) <- oneb' p
              notElemWith sigma x (fv l)
              return ret
         return (sigma,(l, y.\Nu (x.\p')))
  <|> do (x,p) <- unbind b
         (sigma,(Up vy _,p')) <- local (extendCtx (Nab x)) $
          do ret@(sigma,(Up (Var y) (Var x'),_)) <- one p
             equalWith sigma x x' -- guard x==x' under substitution sigma
             noteqWith sigma x y  -- guard x/=y  under substitution sigma
             return ret
         return (sigma,(UpB vy, x.\p')) -- open
oneb _ = empty

oneb' p =
  do (sigma,(l,b)) <- oneb p
     r <- unbind b
     return (sigma,(l,r))

## module OpenBisim

In [33]:
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE FlexibleInstances #-}
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE NoMonomorphismRestriction #-}
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE UndecidableInstances #-}
{-# LANGUAGE PartialTypeSignatures #-}

module OpenBisim where

import Control.Applicative
import Control.Lens.Fold
import Control.Monad
import Control.Monad.Fail
import Control.Monad.Trans.Reader
import qualified Control.Monad.Fail as Fail
import Data.List
import Data.Maybe
import Data.Tree
import Data.Partition hiding (empty,rep)
import qualified Data.Partition as P
import qualified Data.Set as Set
import qualified IdSubLTS as IdS
-- import MemoUgly
import OpenLTS
import PiCalc
import Unbound.Generics.LocallyNameless hiding (fv)

instance MonadFail m => MonadFail (FreshMT m) where
  fail = runFreshMT . Fail.fail

data StepLog = One  Ctx EqC Act  Pr
             | OneB Ctx EqC ActB PrB
             deriving (Eq,Ord,Show)

type EqC = [(Nm,Nm)]

toEqC ctx sigma = [(x,x') | x:xs <- eqcs, x'<-xs]
  where
    ns = quan2nm <$> reverse ctx
    eqcs = map (ns!!) <$> (Set.toList <$> P.nontrivialSets sigma)

stepLog  eitherC ctx sigma l p = Node . eitherC $ One  ctx (toEqC ctx sigma) l p
stepLogB eitherC ctx sigma l b = Node . eitherC $ OneB ctx (toEqC ctx sigma) l b

applySubst m = do (sigma,r) <- m
                  ctx <- ask 
                  return (sigma, subs ctx sigma r)

unbindWith x b = do (y,r) <- unbind b
                    return (x, subst y (Var x) r)


sim ctx p q = and . runFreshMT . (`runReaderT` ctx) $ _sim p q
sim' ctx p q = runFreshMT . (`runReaderT` ctx) $ _sim' p q

bisim ctx p q = and . runFreshMT . (`runReaderT` ctx) $ _bisim p q
bisim' ctx p q = runFreshMT . (`runReaderT` ctx) $ _bisim' p q

_sim = simBool_ id _sim
_sim' = simStepLog_ Left Right id _sim'

_bisim p q = simBool_ id   _bisim p q
         <|> simBool_ flip _bisim q p

_bisim' p q = simStepLog_ Left  Right id   _bisim' p q
          <|> simStepLog_ Right Left  flip _bisim' q p

simBool_ = sim_ or' and' or' and'
  where
    or'  _ _ _ _ = or  :: [Bool] -> Bool
    and' _ _ _ _ = and :: [Bool] -> Bool

simStepLog_ eitherP eitherQ =
    sim_ (stepLog  eitherP) (stepLog  eitherQ)
         (stepLogB eitherP) (stepLogB eitherQ)

sim_ logLeader  logFollow
     logLeaderB logFollowB
     h   -- either id or flip
     rf  -- recursive function call
     p q = 
      do (sigma,(lp,p')) <- applySubst $ one p
         ctx <- ask
         return . logLeader ctx sigma lp p' . runFreshMT $ do
           (lq,q') <- IdS.one (subs ctx sigma q)
           guard $ lp == lq
           return . logFollow ctx sigma lq q' . runFreshMT
                  . (`runReaderT` ctx) $ h rf p' q'
  <|> do (sigma,(lp,bp')) <- applySubst $ oneb p
         ctx <- ask; mapM_ fresh (quan2nm <$> ctx)
         (x,p') <- unbind bp'
         return . logLeaderB ctx sigma lp bp' . runFreshMT $ do
           (lq,bq') <- IdS.oneb (subs ctx sigma q)
           guard $ lp == lq
           (_,q') <- unbindWith x bq'
           let ctx' = case lp of { DnB _ -> All x; UpB _ -> Nab x } : ctx
           return . logFollowB ctx sigma lq bq' . runFreshMT
                  . (`runReaderT` ctx) $ h rf p' q'

forest2df :: [Tree (Either StepLog StepLog)] -> [(Form,Form)]
forest2df rs
            =    do  Node (Left (One _ sigma_p a _)) [] <- rs
                     let sigmaqs = subsMatchingAct a (right1s rs)
                     return (prebase sigma_p a, postbase sigmaqs a)
            <|>  do  Node (Right (One _ sigma_q a _)) [] <- rs
                     let formR = prebase sigma_q a
                     let sigmaps = subsMatchingAct a (left1s rs)
                     return (postbase sigmaps a, formR)
            <|>  do  Node (Left (OneB _ sigma_p a _)) [] <- rs
                     let sigmaqs = subsMatchingActB a (right1Bs rs)
                     return (preBbase sigma_p a, postBbase sigmaqs a)
            <|>  do  Node (Right (OneB _ sigma_q a _)) [] <- rs
                     let formR = preBbase sigma_q a
                     let sigmaps = subsMatchingActB a (left1Bs rs)
                     return (postBbase sigmaps a, formR)
            <|>  do  Node (Left (One _ sigma_p a _)) rsR <- rs
                     let rss' = [rs' | Node _ rs' <- rsR]
                     (dfsL,dfsR) <- unzip <$> sequence (forest2df <$> rss')
                     guard . not . null $ dfsL
                     let sigmaqs = subsMatchingAct a (right1s rs)
                     return (pre sigma_p a dfsL, post sigmaqs a dfsR)
            <|>  do  Node (Right (One _ sigma_q a _)) rsL <- rs
                     let rss' = [rs' | Node _ rs' <- rsL]
                     (dfsL,dfsR) <- unzip <$> sequence (forest2df <$> rss')
                     guard . not . null $ dfsL
                     let sigmaps = subsMatchingAct a (left1s rs)
                     return (post sigmaps a dfsL, pre sigma_q a dfsR)
            <|>  do  Node (Left (OneB nctx sigma_p a _)) rsR <- rs
                     let  rss' = [rs' | Node _ rs' <- rsR]
                          x = quan2nm . head . getCtx . fromEither
                            . rootLabel . head $ head rss'
                     (dfsL,dfsR) <- unzip <$> sequence (forest2df <$> rss')
                     guard . not . null $ dfsL
                     let sigmaqs = subsMatchingActB a (right1Bs rs)
                     return (preB sigma_p a x dfsL, postB sigmaqs a x dfsR)
            <|>  do  Node (Right (OneB nctx sigma_q a _)) rsL <- rs
                     let  rss' = [rs' | Node _ rs' <- rsL]
                          x = quan2nm . head . getCtx . fromEither . rootLabel
                                $ head (head rss')
                     (dfsL,dfsR) <- unzip <$> sequence (forest2df <$> rss')
                     guard . not . null $ dfsL
                     let sigmaps = subsMatchingActB a (left1Bs rs)
                     return (postB sigmaps a x dfsL, preB sigma_q a x dfsR)
  where
    prebase sigma a = pre sigma a []
    postbase sigmas a = post sigmas a []
    preBbase sigma a = preB sigma a (s2n "?") []
    postBbase sigmas a = postB sigmas a (s2n "?") []
    pre sigma a = boxMat sigma . Dia a . conj
    post sigmas a fs = Box a . disj $  (diaMat<$>sigmas) ++ fs
    preB sigma a x = boxMat sigma . DiaB a . bind x . conj
    postB sigmas a x fs = BoxB a . bind x . disj $  (diaMat<$>sigmas) ++ fs
    boxMat  [] = id; boxMat  sigma = BoxMatch [(Var x,Var y) | (x,y)<-sigma]
    diaMat  [] = FF; diaMat  sigma = DiaMatch [(Var x,Var y) | (x,y)<-sigma]
    right1s  rs = [log | Node (Right  log@One{}) _ <- rs]
    left1s   rs = [log | Node (Left   log@One{}) _ <- rs]
    right1Bs  rs = [log | Node (Right  log@OneB{}) _ <- rs]
    left1Bs   rs = [log | Node (Left   log@OneB{}) _ <- rs]
    getCtx (One   nctx _ _ _)  = nctx; getCtx (OneB  nctx _ _ _) = nctx
    fromEither (Left   t) = t; fromEither (Right  t) = t


subsMatchingAct :: Act -> [StepLog] -> [EqC]
subsMatchingAct a logs =
  do  One ctx sigma' a' _ <-logs          ;  let sigmaSubs = subs' ctx sigma'
      guard $ sigmaSubs a == sigmaSubs a' ;  return sigma'

subsMatchingActB :: ActB -> [StepLog] -> [EqC]
subsMatchingActB a logs =
  do  OneB ctx sigma' a' _ <-logs         ;  let sigmaSubs = subs' ctx sigma'
      guard $ sigmaSubs a == sigmaSubs a' ;  return sigma'

subs' ctx eqc = substs [(x,Var y) | (x,y) <- eqc]

# module PP for pretty printing

In [34]:
{-# LANGUAGE FlexibleContexts          #-}
{-# LANGUAGE FlexibleInstances         #-}
{-# LANGUAGE MultiParamTypeClasses     #-}
{-# LANGUAGE NoMonomorphismRestriction #-}
{-# LANGUAGE ScopedTypeVariables       #-}
{-# LANGUAGE UndecidableInstances      #-}
module PP where

import Prelude hiding ((<>))

import           Data.Tree hiding (drawTree, drawForest)
import           Data.Tree.View hiding (drawTree)
import qualified IdSubLTS                       as IdS
import           OpenBisim
import qualified OpenLTS                        as OpS
import           OpenLTS
import           PiCalc
import           Text.PrettyPrint
import           Text.PrettyPrint.HughesPJClass
import           Unbound.Generics.LocallyNameless

import IHaskell.Display
import Data.List (intersperse)

appPrec :: Rational
appPrec = 10

pp = print . pPrint
ppTeX = html . texParen . ppTeXstring

pPrintTeX = pPrint . TeX
ppTeXstring = show . pPrintTeX
texParen s = "$"++s++"$"

instance Pretty Nm where pPrint = text . show

instance Pretty Quan where
  pPrintPrec l r (All x) = maybeParens (r > appPrec) $ text "All" <+> ppp x
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Nab x) = maybeParens (r > appPrec) $ text "Nab" <+> ppp x
    where ppp = pPrintPrec l (appPrec+1)
    
instance (Alpha a, Pretty a) => Pretty (Bind Nm a) where
  pPrintPrec l r b = maybeParens (r > appPrec) $ ppp x <+> text ".\\" <+> ppp p
    where ppp = pPrintPrec l (appPrec+1); (x,p) = runFreshM $ unbind b

instance Pretty Tm where
  pPrintPrec l r (Var x) = maybeParens (r > appPrec) $ text "Var" <+> ppp x
    where ppp = pPrintPrec l (appPrec+1)

instance Pretty Pr where
  pPrintPrec _ _ Null = text "Null"
  pPrintPrec l r (TauP p) = maybeParens (r > appPrec) $
            text "TauP" <+> ppp p
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Out x y p) = maybeParens (r > appPrec) $
            text "Out" <+> ppp x <+> ppp y <+> ppp p
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (In x b) = maybeParens (r > appPrec) $
            text "In" <+> ppp x <+> ppp b
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Plus p q) = maybeParens (r > appPrec) $
            ppp p <+> text "`Plus`" <+> ppp q
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Par p q) = maybeParens (r > appPrec) $
            ppp p <+> text "`Par`" <+> ppp q
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Nu b) = maybeParens (r > appPrec) $
            text "Nu" <> ppp b
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Match x y p) = maybeParens (r > appPrec) $
            text "Match" <+> ppp x <+> ppp y <+> ppp p
    where ppp = pPrintPrec l (appPrec+1)
    

newtype TeX a = TeX a deriving Show

instance (Alpha a, Pretty (TeX a)) => Pretty (TeX(Bind Nm a)) where
  pPrintPrec l r (TeX b) = maybeParens (r > appPrec) $
            text "(" <+> ppp x <+> text ")." <+> ppp p
    where ppp = pPrintPrec l appPrec . TeX; (x,p) = runFreshM $ unbind b

instance Pretty (TeX Nm) where -- assume that we always show free names (Fn{}) only
  pPrintPrec l r (TeX x) = case n of 0 -> text $ s
                                     _ -> text $ s++"_{"++show n++"}"
    where s = name2String x
          n = name2Integer x

instance Pretty (TeX Tm) where
  pPrintPrec l r (TeX (Var x)) = pPrintPrec l r x


plusPrec, parPrec:: Rational
plusPrec = 5
parPrec = 6

instance Pretty (TeX Pr) where
  pPrintPrec _ _ (TeX Null) = text "0"
  pPrintPrec l r (TeX(TauP p)) = maybeParens (r > appPrec) $
            text "\\tau\\!." <+> ppp p
    where ppp = pPrintPrec l appPrec . TeX
  pPrintPrec l r (TeX(Out x y p)) = maybeParens (r > appPrec) $
            text "\\overline" <+> ppp x <+> ppp y <+> text "." <+> ppp p
    where ppp = pPrintPrec l appPrec . TeX
  pPrintPrec l r (TeX(In x b)) = maybeParens (r > appPrec) $
            ppp x <+> ppp b
    where ppp = pPrintPrec l appPrec . TeX
  pPrintPrec l r (TeX(Plus p q)) = maybeParens (r > plusPrec) $
            ppp p <+> text "+" <+> ppp q
    where ppp = pPrintPrec l plusPrec . TeX
  pPrintPrec l r (TeX(Par p q)) = maybeParens (r > parPrec) $
            ppp p <+> text "\\mid{}" <+> ppp q
    where ppp = pPrintPrec l parPrec . TeX
  pPrintPrec l r (TeX(Nu b)) = maybeParens (r > appPrec) $
            text "\\nu" <+> ppp x <+> text "." <+> ppp p
    where ppp = pPrintPrec l appPrec . TeX; (x,p) = runFreshM $ unbind b
  pPrintPrec l r (TeX(Match x y p)) = maybeParens (r > appPrec) $
            text "[" <+> ppp x <+> text "=" <+> ppp y <+> text "]" <+> ppp p
    where ppp = pPrintPrec l appPrec . TeX


instance Pretty Act where
  pPrintPrec l r (Up x y) = maybeParens (r > appPrec) $ text "Up" <+> ppp x <+> ppp y
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r Tau = text "Tau"
instance Pretty ActB where
  pPrintPrec l r (UpB x) = maybeParens (r > appPrec) $ text "UpB" <+> ppp x
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (DnB x) = maybeParens (r > appPrec) $ text "DnB" <+> ppp x
    where ppp = pPrintPrec l (appPrec+1)

instance Pretty Form where
  pPrintPrec _ _ FF = text "FF"
  pPrintPrec _ _ TT = text "TT"
  pPrintPrec l r (Conj fs) = maybeParens (r > appPrec) $
            text "Conj" <> ppp fs
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Disj fs) = maybeParens (r > appPrec) $
            text "Disj" <> ppp fs
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Box a f) = maybeParens (r > appPrec) $
            text "Box" <+> ppp a <+> ppp f
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Dia a f) = maybeParens (r > appPrec) $
             text "Dia" <+> ppp a <+> ppp f
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (BoxB a f) = maybeParens (r > appPrec) $
            text "BoxB" <+> ppp a <+> ppp f
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (DiaB a f) = maybeParens (r > appPrec) $
            text "DiaB" <+> ppp a <+> ppp f
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (BoxMatch sigma f) = maybeParens (r > appPrec) $
            text "BoxMatch" <+> ppp sigma <+> ppp f
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (DiaMatch sigma) = maybeParens (r > appPrec) $
            text "DiaMatch" <+> ppp sigma
    where ppp = pPrintPrec l (appPrec+1)
    
instance Pretty StepLog where
  pPrintPrec l r (One  nctx sigma a p) = maybeParens (r > appPrec) $
            text "One" <+> ppp nctx <+> ppp sigma <+> ppp a <+> ppp p
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (OneB nctx sigma a b) = maybeParens (r > appPrec) $
            text "OneB" <+> ppp nctx <+> ppp sigma <+> ppp a <+> ppp b
    where ppp = pPrintPrec l (appPrec+1)
    
    
    
instance Pretty (TeX Act) where
  pPrintPrec l r (TeX(Up x y)) = maybeParens (r > appPrec) $
            text "\\overline{" <+> ppp x <+> text "}" <+> ppp y
    where ppp = pPrintPrec l (appPrec+1) . TeX
  pPrintPrec l r (TeX Tau) = text "\\tau"
instance Pretty (TeX ActB) where
  pPrintPrec l r (TeX(UpB x)) = maybeParens (r > appPrec) $ text "\\overline{" <+> ppp x <+> text "}"
    where ppp = pPrintPrec l (appPrec+1) . TeX
  pPrintPrec l r (TeX(DnB x)) = maybeParens (r > appPrec) $ ppp x
    where ppp = pPrintPrec l (appPrec+1) . TeX

conjPrec, disjPrec :: Rational
conjPrec = 6
disjPrec = 5


instance Pretty (TeX Form) where
  pPrintPrec _ _ (TeX FF) = text "\\bot{}"
  pPrintPrec _ _ (TeX TT) = text "\\top{}"
  pPrintPrec l r (TeX(Conj [])) = ppp TT
    where ppp = pPrintPrec l conjPrec . TeX  
  pPrintPrec l r (TeX(Conj fs)) = maybeParens (r > conjPrec) $
            foldr1 (<+>) . intersperse (text "\\land{}") $ map ppp fs
    where ppp = pPrintPrec l conjPrec . TeX
  pPrintPrec l r (TeX(Disj [])) = ppp FF
    where ppp = pPrintPrec l disjPrec . TeX
  pPrintPrec l r (TeX(Disj fs)) = maybeParens (r > disjPrec) $
            foldr1 (<+>) . intersperse (text "\\lor{}") $ map ppp fs
    where ppp = pPrintPrec l disjPrec . TeX
  pPrintPrec l r (TeX(Box a f)) = maybeParens (r > appPrec) $
            text "[" <+> ppp a <+> text "]" <+> ppp f
    where ppp = pPrintPrec l appPrec . TeX
  pPrintPrec l r (TeX(Dia a f)) = maybeParens (r > appPrec) $
             text "\\langle{}" <+> ppp a <+> text "\\rangle{}" <+> ppp f
    where ppp = pPrintPrec l appPrec . TeX
  pPrintPrec l r (TeX(BoxB a b)) = maybeParens (r > appPrec) $
            text "[" <+> ppp a <+> text "(" <+> ppp x <+> text ")]" <+> ppp f
    where ppp = pPrintPrec l appPrec . TeX
          (x,f) = runFreshM $ unbind b
  pPrintPrec l r (TeX(DiaB a b)) = maybeParens (r > appPrec) $
            text "\\langle{}" <+> ppp a <+> text "(" <+> ppp x <+> text ")\\rangle{}" <+> ppp f
    where ppp = pPrintPrec l appPrec . TeX
          (x,f) = runFreshM $ unbind b
  pPrintPrec l r (TeX(BoxMatch [] f)) = ppp f
    where ppp = pPrintPrec l appPrec . TeX
  pPrintPrec l r (TeX(BoxMatch sigma f)) = maybeParens (r > appPrec) $
            text "[" <+> foldr1 (<+>) [ppp x <+> text "=" <+> ppp y | (x,y)<-sigma]
        <+> text "]" <+> ppp f
    where ppp = pPrintPrec l appPrec . TeX
  pPrintPrec l r (TeX(DiaMatch [])) = ppp FF
    where ppp = pPrintPrec l appPrec . TeX
  pPrintPrec l r (TeX(DiaMatch sigma)) = maybeParens (r > disjPrec) $
            foldr1 (<+>) $ intersperse (text "\\lor{}") [ppp x <+> text "=" <+> ppp y | (x,y)<-sigma]
    where ppp = pPrintPrec l appPrec . TeX

instance Pretty (TeX StepLog) where
  pPrintPrec l r (TeX(One  nctx sigma a p)) = -- maybeParens (r > appPrec) $
            text "h=" <+> texHistory nctx <+> text ",\\,\\sigma=" <+> ppp sigma <+> text "\\quad"
        <+> text "{-}" <+> tex a <+> text "{\\rightarrow{}}~~" <+> tex p
    where ppp = pPrintPrec l 0; tex = ppp . TeX
  pPrintPrec l r (TeX(OneB nctx sigma a b)) = -- maybeParens (r > appPrec) $
            text "h=" <+> texHistory nctx <+> text ",\\,\\sigma=" <+> ppp sigma <+> text "\\quad"
        <+> text "{-}" <+> tex a <+> text "(\\cdot){\\rightarrow{}}~~" <+> tex b
    where ppp = pPrintPrec l 0; tex = ppp . TeX

texHistory [] = empty
texHistory h  = foldr1 (<+>) . intersperse (text "\\!\\!\\cdot\\!") . reverse $ map texHistoryItem h 
texHistoryItem (All x) = text $ show x++"^i"
texHistoryItem (Nab y) = text $ show y++"^o"

drawTree = showTree . toTreeString
drawForest = unlines . map drawTree

texTree = myHtmlTree . toTreeTeX
texForest = unlines . map texTree

myHtmlTree = show . tree2html

tree2html (Node l ns) = ul . foldr1 (<+>) $ (li . label $ text l) : map tree2html ns
  where
  tag t Nothing content = text("<"++t++">") <+> content <+> text("</"++t++">")
  tag t (Just s) content = text("<"++t++" "++s++">") <+> content <+> text("</"++t++">")
  ul = tag "ul" (Just "class='nested'")
  li = tag "li" Nothing
  label = tag "label" Nothing


render1line = renderStyle style{mode=OneLineMode}

toTreeString = foldTree (\log ts -> Node (render1line . pPrint $ log) ts)
toTreeTeX = foldTree (\log ts -> Node (texParen . render1line . eitherTeX $ log) ts)
  where eitherTeX (Left x) = text "\\color{red}{\\mathsf{L}\\!:" <+> pPrintTeX x <+> text "}"
        eitherTeX (Right y) = text "\\color{blue}{\\mathsf{R}\\!:" <+> pPrintTeX y <+> text "}"

----
## Demo

In [35]:
import qualified IdSubLTS                       as IdS
import           OpenBisim
import qualified OpenLTS                        as OpS
import           OpenLTS
import           PiCalc
import           Text.PrettyPrint
import           Text.PrettyPrint.HughesPJClass
import           Unbound.Generics.LocallyNameless
import PP
import IHaskell.Display

-- variables to use
a = s2n "a"
b = s2n "b"
c = s2n "c"
x = s2n "x"
y = s2n "y"
z = s2n "z"
w = s2n "w"

In [36]:
import Control.Monad.Trans.Reader
import Unbound.Generics.LocallyNameless

-- runBisimExperiment history leftProces rightProcess
runBisimExperiment h prL prR =
  sequence . map (display . html) $
        map (texParen . show)
            [ text "h="<+> texHistory h
            , text "\\mathsf{L}\\!:" <+> pPrintTeX prL
            , text "\\mathsf{R}\\!:" <+> pPrintTeX prR
            , text $ "\\mathsf{L}"
                ++ (if bisimResult then "\\sim" else "\\not\\sim")
                ++ "\\mathsf{R}"
                ++ "\\qquad{}\\text{" ++ (if bisimResult then "" else "non-") ++ "bisimilar}"
            ]
    ++ [ texParen $ "\\phi_L="++ppTeXstring fL++"\\;,~~\\phi_R="++ppTeXstring fR
                                        | (fL,fR) <- forest2df bisimForest ]
    ++ [ texForest bisimForest ]
  where
   bisimResult = bisim h prL prR
   bisimForest = bisim' h prL prR

In [37]:
p = tautau .+ tau
q = (x.=y)tautau .+ tau

ppTeX p
ppTeX q

$\tau\!. \tau\!. 0 + \tau\!. 0$

$[ x = y ] \tau\!. \tau\!. 0 + \tau\!. 0$

In [38]:
:type IdSubLTS.one
:type OpenLTS.one
:type runFreshMT

IdSubLTS.one :: forall (m :: * -> *). (Alternative m, Fresh m, MonadFail m) => Pr -> m (Act, Pr)

OpenLTS.one :: forall (m :: * -> *). (MonadPlus m, Fresh m, MonadFail m) => Pr -> ReaderT [Quan] m (Constraint, (Act, Pr))

runFreshMT :: forall (m :: * -> *) a. Monad m => FreshMT m a -> m a

In [39]:
-- single step transition in the current world
runFreshMT $ IdSubLTS.one q :: [(Act,Pr)]

[(Tau,Null)]

In [51]:
-- single step transition in all possible worlds
runFreshMT . (`runReaderT` (All <$> [x,y])) $ OpenLTS.one q :: [(Constraint,(Act,Pr))]

[(fromDisjointSets [fromList [0,1]],(Tau,TauP Null)),(fromDisjointSets [],(Tau,Null))]

In [52]:
-- runBisimExperiment history leftProces rightProcess
runBisimExperiment (All <$> [x,y]) p q

$h= y^i \!\!\cdot\! x^i$

$\mathsf{L}\!: \tau\!. \tau\!. 0 + \tau\!. 0$

$\mathsf{R}\!: [ x = y ] \tau\!. \tau\!. 0 + \tau\!. 0$

$\mathsf{L}\not\sim\mathsf{R}\qquad{}\text{non-bisimilar}$

$\phi_L=\langle{} \tau \rangle{} \langle{} \tau \rangle{} \top{}\;,~~\phi_R=[ \tau ] (y = x \lor{} [ \tau ] \bot{})$

$\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$

In [53]:
ppTeX p
ppTeX (q .+ tautau)

$\tau\!. \tau\!. 0 + \tau\!. 0$

$[ x = y ] \tau\!. \tau\!. 0 + \tau\!. 0 + \tau\!. \tau\!. 0$

In [54]:
-- runBisimExperiment history leftProces rightProcess
runBisimExperiment (All <$> [x,y]) p (q .+ tautau)

$h= y^i \!\!\cdot\! x^i$

$\mathsf{L}\!: \tau\!. \tau\!. 0 + \tau\!. 0$

$\mathsf{R}\!: [ x = y ] \tau\!. \tau\!. 0 + \tau\!. 0 + \tau\!. \tau\!. 0$

$\mathsf{L}\sim\mathsf{R}\qquad{}\text{bisimilar}$

$\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$

In [55]:
q1 = taup$(x.=y)tau

ppTeX p
ppTeX q1

$\tau\!. \tau\!. 0 + \tau\!. 0$

$\tau\!. [ x = y ] \tau\!. 0$

In [56]:
-- runBisimExperiment history leftProces rightProcess
runBisimExperiment (All <$> [x,y]) p q1

$h= y^i \!\!\cdot\! x^i$

$\mathsf{L}\!: \tau\!. \tau\!. 0 + \tau\!. 0$

$\mathsf{R}\!: \tau\!. [ x = y ] \tau\!. 0$

$\mathsf{L}\not\sim\mathsf{R}\qquad{}\text{non-bisimilar}$

$\phi_L=\langle{} \tau \rangle{} \langle{} \tau \rangle{} \top{}\;,~~\phi_R=[ \tau ] [ \tau ] (y = x)$

$\phi_L=\langle{} \tau \rangle{} [ \tau ] \bot{}\;,~~\phi_R=[ \tau ] [ y = x ] \langle{} \tau \rangle{} \top{}$

$\phi_L=[ \tau ] (\langle{} \tau \rangle{} \top{} \lor{} [ \tau ] \bot{})\;,~~\phi_R=\langle{} \tau \rangle{} ([ \tau ] (y = x) \land{} [ y = x ] \langle{} \tau \rangle{} \top{})$

$\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ [ x = y ] \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ [ x = y ] \tau\!. 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ [ x = y ] \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$

In [57]:
q2 = p .+ q1

ppTeX p
ppTeX q2

$\tau\!. \tau\!. 0 + \tau\!. 0$

$\tau\!. \tau\!. 0 + \tau\!. 0 + \tau\!. [ x = y ] \tau\!. 0$

In [58]:
-- runBisimExperiment history leftProces rightProcess
runBisimExperiment (All <$> [x,y]) p q2

$h= y^i \!\!\cdot\! x^i$

$\mathsf{L}\!: \tau\!. \tau\!. 0 + \tau\!. 0$

$\mathsf{R}\!: \tau\!. \tau\!. 0 + \tau\!. 0 + \tau\!. [ x = y ] \tau\!. 0$

$\mathsf{L}\not\sim\mathsf{R}\qquad{}\text{non-bisimilar}$

$\phi_L=[ \tau ] (\langle{} \tau \rangle{} \top{} \lor{} [ \tau ] \bot{})\;,~~\phi_R=\langle{} \tau \rangle{} ([ \tau ] (y = x) \land{} [ y = x ] \langle{} \tau \rangle{} \top{})$

$\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ [ x = y ] \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ [ x = y ] \tau\!. 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ 
 $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ [ x = y ] \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ \tau\!. 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{red}{\mathsf{L}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [] \quad {-} \tau {\rightarrow{}}~~ 0 }$ $\color{blue}{\mathsf{R}\!: h= y^i \!\!\cdot\! x^i ,\,\sigma= [(y, x)] \quad {-} \tau {\rightarrow{}}~~ 0 }$